<a href="https://colab.research.google.com/github/dellaanima/DLton_Zero_to_One/blob/main/finetune-hfmodel/00_klue_bert_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers -U

In [ ]:
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm -rf ~/.cache/matplotlib

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

Refenrece : https://wikidocs.net/119990

In [ ]:
#  TPU 초기화
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)



In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
import tensorflow as tf

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re

data = pd.read_csv('/content/drive/MyDrive/dlton/data/train.csv')
data.head(20)

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
5,5,직장 내 괴롭힘 대화,나 이틀뒤에 가나다 음식점 예약좀 해줘. 저녁7시로.\n가나다 음식점이요.?\n응....
6,6,기타 괴롭힘 대화,35번 손님 아이스커피 두잔나왔습니다\n아이스커피? \n네 맛있게드세요\n저기요 아...
7,7,기타 괴롭힘 대화,경비아저씨 내차에 경고장 붙였어요?\n내가 여기 몇년째 사는데 아직도 내차인줄 모르...
8,8,기타 괴롭힘 대화,이거 할인 된다면서요\n 확인해보겠습니다\n 아까 저 시람은 할인 해주던데\n 네 ...
9,9,갈취 대화,자네 세금계산서 보내준 거 메일이랑 수치가 다르던데?\n그게.\n얼마나 해먹었어? ...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3950 entries, 0 to 3949
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           3950 non-null   int64 
 1   class         3950 non-null   object
 2   conversation  3950 non-null   object
dtypes: int64(1), object(2)
memory usage: 92.7+ KB


In [ ]:
data.duplicated().sum()

0

In [ ]:
data['class'].value_counts()

기타 괴롭힘 대화      1094
갈취 대화           981
직장 내 괴롭힘 대화     979
협박 대화           896
Name: class, dtype: int64

In [ ]:
def preprocess_sentence(sentence):

    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!가-힣ㄱ-ㅎㅏ-ㅣ]+", " ", sentence)
    sentence = sentence.strip()

    return sentence


In [ ]:

data['conversation'] = data['conversation'].apply(lambda x : preprocess_sentence(x))
data.head(20)


,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가 ? 아닙니다 . 죄송합니다 . 죽을 거...
1,1,협박 대화,길동경찰서입니다 . 시 분 마트에 폭발물을 설치할거다 . 네 ? 똑바로 들어 한번만...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지 ? 나보다 작은 남자는 첨봤어 . 그만해 . 니들 놀리는거 ...
3,3,갈취 대화,어이 거기 예 ? ? 너 말이야 너 . 이리 오라고 무슨 일 . 너 옷 좋아보인다 ...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요 ? 저희 회사에서 이 선크림 파는데 한 번 손등에...
5,5,직장 내 괴롭힘 대화,나 이틀뒤에 가나다 음식점 예약좀 해줘 . 저녁 시로 . 가나다 음식점이요 . ? ...
6,6,기타 괴롭힘 대화,번 손님 아이스커피 두잔나왔습니다 아이스커피 ? 네 맛있게드세요 저기요 아가씨 나는...
7,7,기타 괴롭힘 대화,경비아저씨 내차에 경고장 붙였어요 ? 내가 여기 몇년째 사는데 아직도 내차인줄 모르...
8,8,기타 괴롭힘 대화,이거 할인 된다면서요 확인해보겠습니다 아까 저 시람은 할인 해주던데 네 저 손님은 ...
9,9,갈취 대화,자네 세금계산서 보내준 거 메일이랑 수치가 다르던데 ? 그게 . 얼마나 해먹었어 ?...


Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(data['class'])
num_labels = len(label_encoder.classes_)

data['class'] = np.asarray(label_encoder.transform(data['class']), dtype=np.int32)
data.head(10)


,idx,class,conversation
0,0,3,지금 너 스스로를 죽여달라고 애원하는 것인가 ? 아닙니다 . 죄송합니다 . 죽을 거...
1,1,3,길동경찰서입니다 . 시 분 마트에 폭발물을 설치할거다 . 네 ? 똑바로 들어 한번만...
2,2,1,너 되게 귀여운거 알지 ? 나보다 작은 남자는 첨봤어 . 그만해 . 니들 놀리는거 ...
3,3,0,어이 거기 예 ? ? 너 말이야 너 . 이리 오라고 무슨 일 . 너 옷 좋아보인다 ...
4,4,0,저기요 혹시 날이 너무 뜨겁잖아요 ? 저희 회사에서 이 선크림 파는데 한 번 손등에...
5,5,2,나 이틀뒤에 가나다 음식점 예약좀 해줘 . 저녁 시로 . 가나다 음식점이요 . ? ...
6,6,1,번 손님 아이스커피 두잔나왔습니다 아이스커피 ? 네 맛있게드세요 저기요 아가씨 나는...
7,7,1,경비아저씨 내차에 경고장 붙였어요 ? 내가 여기 몇년째 사는데 아직도 내차인줄 모르...
8,8,1,이거 할인 된다면서요 확인해보겠습니다 아까 저 시람은 할인 해주던데 네 저 손님은 ...
9,9,0,자네 세금계산서 보내준 거 메일이랑 수치가 다르던데 ? 그게 . 얼마나 해먹었어 ?...


In [ ]:
data['class'].value_counts()

1    1094
0     981
2     979
3     896
Name: class, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# conversation : x , label : y
conversation = data['conversation'].tolist()
label = data['class'].tolist()

# train, val 분리 (데이터 0.2)
train_x, val_x, train_y, val_y = train_test_split(conversation, label, random_state=42, test_size=0.2)

In [ ]:
len(train_x), len(val_x)

(3160, 790)

In [ ]:
from transformers import BertTokenizer
HUGGINGFACE_MODEL_PATH = "klue/bert-base"
# Load Tokenizer
tokenizer = BertTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)

# Tokenizing
train_encodings = tokenizer(train_x, truncation=True, padding=True)
val_encodings = tokenizer(val_x, truncation=True, padding=True)

In [ ]:
# trainset-set
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_y
))

# validation-set
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_y
))

In [ ]:
from transformers import TFBertForSequenceClassification


with strategy.scope():
  num_labels = num_labels
  model = TFBertForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=num_labels, from_pt=True)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  110617344 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  3076      
                                                                 
Total params: 110,620,420
Trainable params: 110,620,420
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

callback_earlystop = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.001, # acc가 최소한 얼마만큼의 상승이 있어야하는지 정해줌.
    patience=2)

history = model.fit(
    train_dataset.shuffle(len(train_dataset)).batch(16), epochs=5, batch_size=16,
    validation_data=val_dataset.shuffle(len(val_dataset)).batch(16),
    callbacks = [callback_earlystop]
)


Epoch 1/5
198/198 [==============================] - 134s 202ms/step - loss: 0.5035 - accuracy: 0.8184 - val_loss: 0.2992 - val_accuracy: 0.9051
Epoch 2/5
198/198 [==============================] - 28s 142ms/step - loss: 0.1854 - accuracy: 0.9421 - val_loss: 0.4036 - val_accuracy: 0.8759
Epoch 3/5
198/198 [==============================] - 29s 147ms/step - loss: 0.0944 - accuracy: 0.9696 - val_loss: 0.3653 - val_accuracy: 0.9013


In [ ]:


id2labels = model.config.id2label
model.config.id2label = {id : label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0]  for id, label in id2labels.items()}

label2ids = model.config.label2id
model.config.label2id = {label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0] : id   for id, label in id2labels.items()}



In [ ]:
import os
DIR_PATH = '/content/drive/MyDrive/dlton/model'
MODEL_NAME = 'fine-tuned-klue-bert-base'
MODEL_SAVE_PATH = os.path.join(DIR_PATH, MODEL_NAME) # change this to your preferred location

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")
else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

# save tokenizer, model
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

/content/drive/MyDrive/dlton/model/fine-tuned-klue-bert-base -- Folder create complete 



('/content/drive/MyDrive/dlton/model/fine-tuned-klue-bert-base/tokenizer_config.json',
 '/content/drive/MyDrive/dlton/model/fine-tuned-klue-bert-base/special_tokens_map.json',
 '/content/drive/MyDrive/dlton/model/fine-tuned-klue-bert-base/vocab.txt',
 '/content/drive/MyDrive/dlton/model/fine-tuned-klue-bert-base/added_tokens.json')

In [ ]:
from transformers import TextClassificationPipeline

# Load Fine-tuning model
loaded_tokenizer = BertTokenizer.from_pretrained(MODEL_SAVE_PATH)
loaded_model = TFBertForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

text_classifier = TextClassificationPipeline(
    tokenizer=loaded_tokenizer,
    model=loaded_model,
    framework='tf',
    top_k=None
)

Some layers from the model checkpoint at /content/drive/MyDrive/dlton/model/fine-tuned-klue-bert-base were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/dlton/model/fine-tuned-klue-bert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further

In [ ]:


# Load Test-set
test_file_path = '/content/drive/MyDrive/dlton/data/test.json'
with open(test_file_path, mode='rt', encoding='utf-8') as f:
    test_dataset = pd.read_json(f)
test_dataset



,t_000,t_001,t_002,t_003,t_004,t_005,t_006,t_007,t_008,t_009,...,t_490,t_491,t_492,t_493,t_494,t_495,t_496,t_497,t_498,t_499
text,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...,그러니까 빨리 말해. 선생님 제발 살려주십시오. 비밀번호 틀릴 때마다 손톱 하나씩...,그 사람 누구냐? 누구말하는거야? 다 알면서 모른척 하지마. 둘다 쏴버리기 전에...,야 저기야 닭꼬치 트럭왔다 응 그러네 그치? 너도 먹고 싶지? 어?나도? 그래 너 ...,안경 안쓰고 키작고통통해보이는 분이었는데 다안경안씀 쌍꺼풀없었던것같고 아 단발인가...,야 너 2학년 김민석 맞지? 네 맞는데요. 혹시 누구신가요? 내가 누군지 궁금하면 ...,...,철수야 말 들어야지 싫은데 철밥통 선생아 너 선생님한테 말버릇이 뭐야 야 내가 너말...,"양재 터널 입구 앞인데요. 사고가 났거든요? 위치가 양재 터널 맞으시죠? 네, 지금...",동생이쁘다. 나주라 안돼 우리동생건들지마. 왜 안되냐? 좀 달라고 니 목숨값으로 동...,혹시 나 오늘 2만원만 빌려주만 안돼? 응? 안돼. 나도 요즘 맨날 택시타느라 택시...,이봐 네? 누구세요? 입다물고 가진거 다내놔 경찰에 신고할거에요 할 수 있으면 해봐...,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...,엄마 저 그 돈 안해주시면 정말 큰일나요. 이유도 말하지 않고. 몇번째니 경민아....


In [ ]:
test_data = test_dataset.transpose()

In [ ]:
test_data

,text
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...
...,...
t_495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...
t_496,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...
t_497,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...
t_498,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...


In [ ]:
preds_list = (text_classifier(test_data['text'][388]))[0]
sorted_preds_list = sorted(preds_list, key=lambda x: x['score'], reverse=True)
print(sorted_preds_list)

[{'label': '협박 대화', 'score': 0.9238980412483215}, {'label': '기타 괴롭힘 대화', 'score': 0.03657250478863716}, {'label': '직장 내 괴롭힘 대화', 'score': 0.02191716432571411}, {'label': '갈취 대화', 'score': 0.017612313851714134}]


In [ ]:
predicted_label_list = []
predicted_score_list = []

for text in test_data['text']:
    # predict
    preds_list = text_classifier(text)[0][0]
    predicted_label_list.append(preds_list['label']) # label
    predicted_score_list.append(preds_list['score']) # score

In [ ]:
submission = test_data.assign(CLASS=predicted_label_list)
submission = submission.rename(columns={'CLASS':'class'})

In [ ]:
submission.head(10)

,text,class
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...,갈취 대화
t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...,직장 내 괴롭힘 대화
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...,직장 내 괴롭힘 대화
t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...,기타 괴롭힘 대화
t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...,기타 괴롭힘 대화
t_005,그러니까 빨리 말해. 선생님 제발 살려주십시오. 비밀번호 틀릴 때마다 손톱 하나씩...,협박 대화
t_006,그 사람 누구냐? 누구말하는거야? 다 알면서 모른척 하지마. 둘다 쏴버리기 전에...,협박 대화
t_007,야 저기야 닭꼬치 트럭왔다 응 그러네 그치? 너도 먹고 싶지? 어?나도? 그래 너 ...,갈취 대화
t_008,안경 안쓰고 키작고통통해보이는 분이었는데 다안경안씀 쌍꺼풀없었던것같고 아 단발인가...,직장 내 괴롭힘 대화
t_009,야 너 2학년 김민석 맞지? 네 맞는데요. 혹시 누구신가요? 내가 누군지 궁금하면 ...,갈취 대화


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
submission.drop(['text'], axis=1, inplace=True)

In [ ]:

submission


,t_000,t_001,t_002,t_003,t_004,t_005,t_006,t_007,t_008,t_009,...,t_490,t_491,t_492,t_493,t_494,t_495,t_496,t_497,t_498,t_499
class,0,2,2,1,1,3,3,0,2,0,...,1,3,3,0,0,2,2,0,2,0


갈취 대화 , 직장 내 괴롭힘 대화, 협박 대화, 기타 괴롭힘 대화





In [ ]:
# Sample class mapping
class_mapping = {
    '협박 대화': 00,
    '갈취 대화': 01,
    '직장 내 괴롭힘 대화': 02,
    '기타 괴롭힘 대화': 03
}


In [ ]:
# Apply Label Encoding
label_encoder = LabelEncoder()
submission['class'] = label_encoder.fit_transform(submission['class'])

In [ ]:
submission = submission.transpose()

In [ ]:
submission

,class
t_000,0
t_001,2
t_002,2
t_003,1
t_004,1
...,...
t_495,2
t_496,2
t_497,0
t_498,2


In [ ]:
import json

submission_file_path = '/content/drive/MyDrive/dlton/submission_01.json'
result = submission.to_json(submission_file_path)

with open(submission_file_path) as f:
    parsed = json.load(f)

with open(submission_file_path, 'w') as f:
    json.dump(parsed, f, indent=4)

In [ ]:
import pandas as pd

# Assume 'submission' is a DataFrame containing the submission data
submission_file_path = '/content/drive/MyDrive/dlton/submission_01.csv'
submission.to_csv(submission_file_path, index=True)

# You can load and verify the saved CSV file if needed
parsed = pd.read_csv(submission_file_path)

print(parsed.head())  # Display the first few rows of the loaded CSV data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
submission_file_path = '/content/drive/MyDrive/dlton/submission_01(1).csv'

In [ ]:
import pandas as pd
df = pd.read_csv(submission_file_path)
print(df)

    file_name  class
0       t_000      0
1       t_001      2
2       t_002      2
3       t_003      1
4       t_004      1
..        ...    ...
495     t_495      2
496     t_496      2
497     t_497      0
498     t_498      2
499     t_499      0

[500 rows x 2 columns]


In [ ]:
# class 칼럼의 값들을 00, 01, 02, 03 으로 변환합니다.
df["class"] = df["class"].replace({0: "00", 1: "01", 2: "02", 3: "03"})

In [ ]:
df

,file_name,class
0,t_000,00
1,t_001,02
2,t_002,02
3,t_003,01
4,t_004,01
...,...,...
495,t_495,02
496,t_496,02
497,t_497,00
498,t_498,02


In [ ]:
# df 객체를 csv 파일로 저장합니다.
df.to_csv("/content/drive/MyDrive/dlton/submission_01(2).csv", index=False)